In [54]:
#get document frequency for terms in the same article/instance; and get each sentence's BOW; and get each question's BOW;
#get each sentence's length; get each article's average length of sentences
import csv
import json
import copy
import os
import nltk
from math import log
from collections import defaultdict, Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance
from sklearn.decomposition import TruncatedSVD
from nltk.tag import StanfordNERTagger
def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
def input_data():
    base_path = os.path.join('')
    train_file = base_path + 'QA_train.json'
    train_data = json.load(open(train_file))
    test_file = base_path + 'QA_test.json'
    test_data = json.load(open(test_file))
    dev_file = base_path + 'QA_dev.json'
    dev_data = json.load(open(dev_file))

    return train_data,test_data,dev_data
def transform_text(text):
    text = nltk.word_tokenize(text)
    text = [lemmatize(word.lower()) for word in text]
    result = []
    for word in text:
        if word not in stopwords and word not in punctuations:
            result.append(word)
    return result
def get_Docfrequency_SentenceBOW(dataset):
    #save dics, each dictionary contains document frequencies for all terms in the same article
    question_list = []
    #save lists, each list represent an article, saving sentences' bow
    total_sentence_bow = []
    #save lists, each list represent an article, saving questions' bow
    total_question_bow = []
    #save lists, each list represent all sentences' lengthes.
    sent_lengthes = []
    #save a list, each item represents the average length of sentences
    avg_lengthes = []
    #
    answer_id = []
    
    for article in dataset:
        #Docfrequency
        article_dic = defaultdict(list)
        keyterms = [] #save all distinct terms in questions
        
        #SentenceBOW
        bow_list = []
        
        #QuestionBOW
        que_list = []
        
        #SentenceLength
        sent_len = []
        
        #TotalLength
        total_len = 0
        
        #RightAnser
        right_answer = []
        
        qas = article['qa']
        sentences = article['sentences']
        for qa in qas:
            question = qa['question']
            newquestion = transform_text(question)
            #QuestionBOW
            que_list.append(get_BOW(newquestion))
            answer = qa['answer_sentence']
            right_answer.append(answer)
            
            keyterms.extend(newquestion)
        keyterms = set(keyterms)
        
        #save sentences' BOW in list sen_BOW
        sen_words = []
        for sent in sentences:
            sent = transform_text(sent)
            #Docfrequency
            sen_words.append(sent)
            
            #SentenceBOW
            bow_list.append(get_BOW(sent))
            
            #SentenceLength
            sent_len.append(len(sent))
            
            #TotalLength
            total_len += len(sent)
            
        
        #calculate doc frequency    
        for term in keyterms:
            for i,bow in enumerate(sen_words):
                if term in bow:
                    article_dic[term].append(i)
                    
        #Docfrequency
        question_list.append(article_dic)
        #SentenceBOW
        total_sentence_bow.append(bow_list)
        #QuestionBOW
        total_question_bow.append(que_list)
        #SentenceLength
        sent_lengthes.append(sent_len)
        #AverageLength
        avg_lengthes.append(float(total_len)/len(sentences))
        #
        answer_id.append(right_answer)
        
    return question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id

train,test,dev = input_data()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
punctuations = [',','\'\'','?','\'','.','%','(',')',';','``']
question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id_list= get_Docfrequency_SentenceBOW(train)

In [55]:
import math
def BM25(articles_index,k1,k2,b):
    total_queries = len(total_question_bow[articles_index])
    count = 0
    for index in range(len(total_question_bow[articles_index])):
        answer_id = answer_id_list[articles_index][index]
        guess_id = find_max_score_sentence(articles_index,index,k1,k2,b)
        if answer_id == guess_id:
            count += 1   
    accurancy = float(count)/total_queries  
    return accurancy

def find_max_score_sentence(articles_index,index,k1,k2,b):

    query_dict = total_question_bow[articles_index][index]
    max_score = 0
    guess_sentence = 0
    for index in range(len(total_sentence_bow[articles_index])):     
        score = 0  
        sentence_dict = total_sentence_bow[articles_index][index]
        for word in query_dict:
            document_fre_list = question_list[articles_index].get(word,None)
            
            
            N = len(total_sentence_bow[articles_index])
            n_qi = 0
            if document_fre_list != None:
                n_qi = len(document_fre_list)
            else:
                n_qi = 0
            fi = sentence_dict.get(word,0)
            qfi = query_dict.get(word,0)
            dl = sent_lengthes[articles_index][index]
            avgdl = avg_lengthes[articles_index]
            
            K = k1*(1-b+b*(float(dl)/avgdl)) 
            W = math.log((N-n_qi+0.5)/(n_qi+0.5))
            R = (fi*(k1+1))/(fi+K)*qfi*(k2+1)/(qfi+k2)
            score += W*R
        if score > max_score:
            max_score = score
            guess_sentence = index
    return guess_sentence

In [56]:
accurancy = 0.0

k1_list = [1.1]
k2_list = [110]
b_list = [0.18]

test_length = len(train)
for k1 in k1_list:
    for k2 in k2_list:
        for b in b_list:
            accurancy = 0.0
            for i in range(0,test_length):
                i_accurancy = BM25(i,k1,k2,b)
                accurancy += i_accurancy
            average_accurancy = accurancy/test_length
            print "Accurancy of BM25 with k1:",k1," k2:",k2," b:",b
            print average_accurancy



Accurancy of BM25 with k1: 1.1  k2: 110  b: 0.18
0.683486756477


In [ ]:
#Accurancy of BM25 with k1: 1.1  k2: 110  b: 0.18
#0.683486756477

In [66]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import re
import sys
def token(text):
    result = []
    combo = u''
    flag_upper = False
    x = re.split(r'("|\.|\?|;|,|\(|\s|\))\s*', text)
    #print x
    for word in x:
        if word != u'' and word != u' ':
            if word[0].isalpha():
                if flag_upper == False and word[0].isupper():
                    combo = word
                    flag_upper = True
                elif flag_upper == False and word[0].islower():
                    result.append(word)
                elif flag_upper == True and word[0].isupper():
                    combo = combo + u' ' + word
                elif flag_upper == True and word[0].islower():
                    result.append(combo)
                    result.append(word)
                    flag_upper = False
            else:
                if flag_upper == True:
                    result.append(combo)
                    result.append(word)
                    flag_upper = False
                else:
                    result.append(word)
    return result
def transfer_pos_question(pos):
    new_pos = []
    for (word,wtype) in pos:
        if word.lower() == 'what':
            new_pos.append((word,'WHAT'))
        elif word.lower() == 'do' or word.lower() == 'does' or word.lower() == 'did':
            new_pos.append((word,'DO'))
        elif word.lower() == 'year' or word.lower() == 'era' or word.lower() == 'ration' or word.lower() == 'years' or word.lower() == 'time':
            new_pos.append((word,'TIME'))
        elif word.lower() == 'is' or word.lower() == 'was' or word.lower() == 'are' or word.lower() == 'were':
            new_pos.append((word,'BE'))
        else:
            new_pos.append((word,wtype))
    return new_pos
def transfer_pos_sentence(pos):
    new_pos = []
    for (word,wtype) in pos:
        if word.lower() == 'and' or word.lower() == 'or':
            new_pos.append((word,'POSICC'))
        elif word.lower() == 'with':
            new_pos.append((word,'WITH'))
        elif word.lower() == 'a' or word.lower() == 'an':
            new_pos.append((word,'A'))
        elif word == '"':
            new_pos.append((word,'"'))
        else:
            new_pos.append((word,wtype))
    return new_pos
def get_continuous_chunks(text,texttype):
    #text = text.encode('ascii','replace')
    t = copy.deepcopy(text)
    #print token(t)
    pos =  pos_tag(token(t))
    if texttype==0:
        pos = transfer_pos_sentence(pos)
        #print pos
        #<\(>{<[^\(\)]*>+}<\)>
        grammar = r"""
                    J:
                        {<JJ.*><VBN>}
                        {<JJ.*><POSICC><JJ.*>}   
                        {<JJ.*>+}
                        {<NN.*><POS>}
                    N:
                        {<A>?<J>?<NN.*>+}
                        <\(>{(<CD><,>)*<CD>}<\)>
                        <\">{<A>?<J>?<NN.*>+}<\">
                    COMBON:
                        {(<N><,>)*<N><,>?<POSICC><N>}
                    NWC:
                        {<N><WITH><COMBON>}
                    """
    elif texttype==1:
        pos = transfer_pos_question(pos)
        #print pos
        grammar = r"""
                    WHATDO:
                        {<WHAT><DO>}
                    TIME:
                        {<WHAT><BE>?<DT>?<NN>?<JJ>?<TIME>}
                    WHAT: 
                        {<WHAT>}
                    """
    cp = nltk.RegexpParser(grammar) 
    result = []
    poss = copy.deepcopy(pos)
    tree = cp.parse(pos)
    #record the position of pos
    flag = 0
    for subtree in tree.subtrees():
        if texttype==0 and subtree.label() != 'S':
            phrase = u''
            for word,pos in subtree.leaves():
                if word == ',':
                    phrase = phrase + word
                else:
                    phrase = phrase + u' '
                    phrase = phrase + word
            result.append((subtree.label(),phrase[1:]))
            #print subtree.label(),phrase
        elif texttype==1 and subtree.label() != 'S':
            phrase = u' '.join([word for word,pos in subtree.leaves()])
            result.append((subtree.label(),phrase))
            #print subtree.label(),phrase
            
    return poss,result




Question: What does LP stand for when it comes to time capacity?
Question: What are common diameters found in phonograph records?
Question: What is a gramophone or 'vinyl' record?
Question: Where does the groove on a vinyl record typically start?
Question: In what method is the rotational speed measured in?
Question: What was the primary use of a phonographic disc record?
Question: Which year did vinyl records leave the main steam media market?
Question: From the 1990s to 2010s who was the primary consumer of vinyl records?
Question: Approximately how many phonograph records were sold in 2014?
Question: What is the niche market of phonograph record fans known as?
Question: What was the original intent of the phonautograph?
Question: In what years where phonautograms converted to audible sound?
Question: What year were the earliest known recordings of sound?
Question: By whom was the phonautograms patented?
Question: In what year was phonautograms patented?
Question: In what year did Th

In [ ]:
from nltk.tag import StanfordNERTagger
def input_NER():
    stanford_dir = os.path.join('stanford-ner-2016-10-31')
    jarfile = os.path.join(stanford_dir,'stanford-ner.jar')
    modelfile = os.path.join(stanford_dir,'classifiers/english.muc.7class.distsim.crf.ser.gz')
    return modelfile,jarfile
model,jar = input_NER()
st = StanfordNERTagger(model,jar)

total_count = 0.0
total_question = 0.0
#for index,article in enumerate(train[:1]):
article = train[0]
index = 0
qas = article['qa']
sentences = article['sentences']
count = 0
for i in correct[index]:
    qa = qas[i]
    #print '---------------'
    flag = False
    text_question = qa['question']
    answer_sentence = sentences[qa['answer_sentence']]
    pos,result = get_continuous_chunks(text_question,1)
    #print pos
    if result != []:
        wtype,word = result[0]
        if wtype == 'WHAT':
            answer_list = []
            total_question += 1
            print '---------------'
            #print result
            wpos,wresult = get_continuous_chunks(answer_sentence,0)
            for wwtype,wword in wresult:
                if qa['answer'] == wword:
                    flag = True
                    total_count += 1
            if flag == True:
                #print 'Q: ',text_question
                #print 'S: ',answer_sentence
                print 'A: ',qa['answer']
                for t,w in wresult:
                    if t != u'J':
                        answer_list.append(w)
                print answer_list
print total_count/total_question